<a href="https://colab.research.google.com/github/tysonjohn015/P04_AI_Credit_Risk_Model/blob/main/P4_01_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJET 4 - CONSTRUISEZ UN MODELE DE SCORING

La société financière "Prêt à dépenser" souhaite pouvoir utiliser un modèle de scoring l'aidant à prédire le risque de défaut de paiement d'un client ayant peu ou pas d'historique de prêt.  
Le modèle devra permettre aux conseillers qui l'utilisent de comprendre les motifs de l'acceptation ou du rejet de la demande de prêt.  

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 200, 'display.max_rows', None, 'display.max_columns', None)

import sklearn
import plotly
from distutils.version import LooseVersion as version
if version(sklearn.__version__) < version("0.24.1"):
    !pip install --upgrade plotly
    print("Sorry, I need at least version 0.24.1 of sklearn.")
else:
    from sklearn import set_config
    set_config(display='diagram')
if version(plotly.__version__) < version("4.14.3"):
    !pip install --upgrade scikit-learn
    print("Sorry, I need at least version 4.14.3 of plotly")

import numpy as np
import os
from tqdm import tqdm
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, learning_curve
from sklearn.metrics import plot_confusion_matrix, fbeta_score, make_scorer, accuracy_score, confusion_matrix, plot_roc_curve
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier

print('Imports terminés.')

## 1. Exploration des données

Télécharger les données de kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# install the Kaggle API client.
!pip install -q kaggle
! mkdir ~/.kaggle

!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

! chmod 600 ~/.kaggle/kaggle.json

# Copy the stackoverflow data set locally.
!kaggle datasets download -d tysonjohn/datadd -f application_train.csv
# !kaggle datasets download -d tysonjohn/datadd -f application_test.csv
# !kaggle datasets download -d tysonjohn/datadd -f bureau.csv
# !kaggle datasets download -d tysonjohn/datadd -f bureau_balance.csv
# !kaggle datasets download -d tysonjohn/datadd -f previous_application.csv
# !kaggle datasets download -d tysonjohn/datadd -f POS_CASH_balance.csv
# !kaggle datasets download -d tysonjohn/datadd -f installments_payments.csv
# !kaggle datasets download -d tysonjohn/datadd -f credit_card_balance.csv

!unzip application_train.csv.zip
# !unzip application_test.csv.zip
# !unzip bureau.csv.zip
# !unzip bureau_balance.csv.zip
# !unzip previous_application.csv.zip
# !unzip POS_CASH_balance.csv.zip
# !unzip installments_payments.csv.zip
# !unzip credit_card_balance.csv.zip

Les données sont réparties en plusieurs fichiers.

### 1.1 Quel(s) fichier(s) utiliser ?

Sur les 9 fichiers fournis (hors celui détaillant les informations contenues dans les autres) :

- **application_train.csv** contient des données relatives à l'emprunteur lui-même, et concernant son crédit, au sort de celui-ci (mis en défaut de paiement ou non). Puisque le modèle est destiné à évaluer le risque de défaut d'emprunteurs n'ayant pas d'historique de crédit, il est bien sûr à utiliser.

- **application_test.csv** ne nous sera pas utile dans le présent cas, car il ne comporte pas d'étiquettes relatives au sort du prêt (défaut ou pas). On ne l'utilisera donc pas.

- **POS_CASH_balance.csv** : ce fichier est relatif au prêt en cours et au prêt précédent. Les données ne seront donc pas disponibles pour un nouvel emprunteur : il n'est donc pas utile pour l'entraînement du modèle.

- **bureau.csv** : contient des informations sur le prêt en cours, donc là encore, inutile vu notre objectif.

- **bureau_balance.csv** : La description précise "use this to join to CREDIT_BUREAU table", qui n'apparaît pas dans la liste. On prendra donc l'hypothèse qu'il faut la joindre avec la table bureau. Cette dernière n'étant pas utilisée, la présente table ne sera pas non plus utile.

- **credit_card_balance** : se réfère au prêt en cours et au prêt précédent, donc inutile

- **installment_payments** : idem

- **previous_application_csv**: idem 

- **sample_submission.csv** : la signification de ce ficher n'est pas évidente puisque toutes les valeurs TARGET sont à 0.5. Faute d'information sur son utilité, il ne sera pas utilisé.

Au final, seules les données contenues dans application_train.csv seront utilisées pour l'entraînement du modèle.

In [ ]:
application_train = pd.read_csv('application_train.csv')
application_train.head()

## 2. Nettoyage des données

Pour optimiser le pipeline pour une utilisation future, la phase de nettoyage séquentiel des données est divisée en deux activités qui s'améliorent au fur et à mesure que le processus progresse.

- une fonction destinée à supprimer les variables éventuellement redondantes et à créer de nouvelles variables synthétiques

- une fonction destinée à traiter les valeurs manquantes ou aberrantes.

En première approche, nous allons commencer par entraîner un modèle de classification simple de type arbre de décision, sur les données simplement nettoyées des valeurs vides ou aberrantes. On ne modifiera donc pas les variables elles-mêmes dans cette étape. Puis, après examen des premiers résulats, nous affinerons notre démarche en supprimant / créant des variables si nécessaire. C'est donc la fonction de traitement des valeurs manquantes / aberrantes qui va être créée en premier lieu.

### 2.1 Recherche des anomalies de valeurs numériques

Etudions les distributions pour les variables numériques.

In [ ]:
data = application_train.copy()

In [ ]:
def plot_hist_data(df):
    num_cols = [col for col in df.columns if data[col].dtype != 'object'] 
        
    height = int(np.ceil(len(num_cols)/6))
    fig_height = 3 * height
    fig = plt.figure(figsize=(20,fig_height))
    
    for feat_idx, col in enumerate(num_cols):
        ax = fig.add_subplot(height, 6, feat_idx+1)
        ax.hist(df[col], bins=50)
        ax.set_title(col)
    fig.tight_layout(pad=4)  

In [ ]:
def plot_box_data(df):
    num_cols = [col for col in df.columns if data[col].dtype != 'object'] 
    
    height = int(np.ceil(len(num_cols)/6))
    fig_height = 3 * height
    fig = plt.figure(figsize=(20,fig_height))
    
    for feat_idx, col in enumerate(num_cols):
        ax = fig.add_subplot(height, 6, feat_idx+1)
        ax.boxplot(df[col])
        ax.set_title(col)
    fig.tight_layout(pad=4)  

In [ ]:
# plot_hist_data(data)

In [ ]:
# plot_box_data(data)

#### AMT_INCOME_TOTAL
On observe des valeurs irréalistes :

In [ ]:
px.box(data['AMT_INCOME_TOTAL'])

Pour écarter les valeurs aberrantes, on utilisera le Z-score, c'est-à-dire la distance à la moyenne divisée par l'écart-type.  

Si Z-score > 3, la valeur peut être considérée comme un outlier puisqu'elle ne fait pas partie des 99,7% valeurs les plus proches de la moyenne.

Remarquons toutefois qu'en valeur absolue, le nombre de cas potentiellement écartés n'est pas négligeable :

In [ ]:
def z_score(array, threshold=3):
    '''Return an array of boolean, True for each value in the array where Z-score >threshold.'''
    mean = array.mean()
    std = array.std()
    return abs((array-mean))/std > threshold

In [ ]:
nb_outliers = data[z_score(data['AMT_INCOME_TOTAL'])].shape[0]
pc_outliers = nb_outliers / data.shape[0]
print(f'Concernant la variable AMT_INCOME_TOTAL, on retrouve {nb_outliers} outliers \
qui représentent {pc_outliers:.2%} des cas.')

On risque donc de se priver d'outliers qui ne sont pas forcément des valeurs aberrantes, mais simplement hors norme, et qui pourraient obérer l'efficacité du modèle pour les très hauts revenus. Examinons les 10 plus gros revenus de la liste des emprunteurs :

In [ ]:
data.sort_values(by='AMT_INCOME_TOTAL', ascending=False).head(10).style.format({'AMT_INCOME_TOTAL':'{:,.0f}'})

Mais le modèle que nous devons concevoir est fait pour être généraliste et s'appliquer aux cas 'normaux', au sens propre du terme. C'est donc en connaissance de cause que nous écarterons les données avec un Z-score supérieur à 3.  

Il faudra garder à l'esprit que le modèle risque donc de ne pas donner de bons résultats sur les cas extrêmes (très hauts revenus notamment), des cas qui nécessiteront sans doute une approche manuelle, ou un autre modèle.

Les autres variables étant susceptibles d'être supprimées ou modifiées par la suite, on appliquera un traitement global avec le même seuil de Z-score à 3.

#### Données en nombre de jours
Convertissons-les temporairement en années écoulées (divison par -365) pour faciliter la vérification : 

In [ ]:
px.box(data['DAYS_EMPLOYED'])

In [ ]:
day_cols = [col for col in data.columns if "DAYS_" in col]
(data[day_cols].describe().loc[['min', 'max']].transpose()/-365).style.format({'min':'{:,.2f}', 'max':'{:,.2f}'})

Le max de 1000 ans pour DAYS_EMPLOYED est visible sur l'histogramme (sous la valeur -365 000 environ) et correspond à un nombre non négligeable de lignes (on affiche cette fois-ci les valeurs d'origine, sans la division par -365):

In [ ]:
(data[day_cols])[data['DAYS_EMPLOYED']>0].describe()

Les valeurs sont toutes indentiques à 365 243 jours. Cette valeur aberrante et constante suggère un parti pris dans la saisie ou l'encodage, par exemple dans le cas où la durée dans l'emploi n'est pas connue ou parce que la zone n'est pas remplie.

In [ ]:
days_count_pos = data['DAYS_EMPLOYED'][data['DAYS_EMPLOYED']>0].count()
print(f"Nombre de lignes avec des valeurs aberrantes pour DAYS_EMPLOYED : {days_count_pos} soit {days_count_pos/data.shape[0]:.2%} du total")

Le nombre de cas concernés est trop important pour supprimer purement et simplement les lignes correspondantes. Nous allons prendre le parti de transformer ces valeurs en la valeur réaliste la plus proche, soit 0. Aux yeux d'un banquier, ne pas avoir d'emploi est à peu près équivalent à démarrer dans un poste le jour même de la demande de prêt.

Par ailleurs, des durées supérieures à l'âge seraient également aberrantes, vérifions si c'est le cas :

In [ ]:
for col  in day_cols:
    idx = data[data[col]<data['DAYS_BIRTH']].index
    idx_exist = len(idx)
    if idx_exist:
        print(col, 'présente des valeurs aberrantes aux lignes :', list(idx))
        for i in idx:
            print(data.loc[i,[col, 'DAYS_BIRTH']])

Mis à part cette erreur liée à un simple arrondi, les autres valeurs ne semblent pas aberrantes à cet égard.

On établit donc comme critères de valeur aberrantes :
- les valeurs > 0 qui seront ramenées à 0
- pour toutes les valeurs sauf DAYS_BIRTH, des valeurs inférieures à celle-ci (autrement dit des durées supérieures à l'âge), qui seront ramenées à la valeur de DAYS_BIRTH.

On pourrait aussi fixer des limites hautes et basse à DAYS_BIRTH, mais il est possible que la législation bancaire américaine permette les emprunts par des mineurs ou des personnes de plus de 70 ans : faute d'information, et les données présentes ne montrant pas d'aberration, on ne modifiera donc pas cette variable.

In [ ]:
data[data['DAYS_EMPLOYED']>0].head()

In [ ]:
data.loc[[8, 11, 23, 266366],day_cols]

In [ ]:
for col in day_cols:
    data.loc[data[col]< data['DAYS_BIRTH'], col] = data['DAYS_BIRTH']
    data.loc[data[col]>0, col] = 0

In [ ]:
data.loc[[8, 11, 23, 266366],day_cols]

#### Valeurs quantitatives ou qualitatives nulles ou manquantes

In [ ]:
import seaborn as sns
plt.figure(figsize=(20,10))
sns.heatmap(data.isna(),cbar=False)
plt.title("Répartition des valeurs vides")
plt.show()

In [ ]:
print(data.dtypes.value_counts())
plt.figure(figsize=(12,10))
plt.pie(data.dtypes.value_counts().values,autopct="%1.2f%%",labels=[str(types) for types in data.dtypes.value_counts().index])
plt.title("Répartition des types dans le jeu de données.")
plt.ylabel("Type des données")
plt.legend()
plt.show()

Examinons les valeurs pour les variables qualitatives :

In [ ]:
for col in [col for col in data.columns if data[col].dtype == 'object']:
    print(col,":", list(application_train[col].unique()))

Les données correspondent à des historiques de prêt réellement accordés. Les éventuelles valeurs nulles constatées sont donc en principe "réelles" : soit parce qu'il s'agit de valeurs binaires (FLAGS), soit parce que l'information est effectivement égale à zéro (ex : montant du salaire) ou indisponible (valeur 'nan') dans le dossier. 

Il ne nous paraît pas possible dans ce dataset de procéder à une imputation, chaque dossier étant unique. Mais les modèles n'acceptant pas la valeur NaN, si une donnée n'est pas connue, elle sera considérée comme nulle ou vide (selon sa nature) lors de la normalisation.

Précisons que le choix de remplacer les valeurs quantitatives NaN à zéro peut se discuter. D'un point de vue métier, il se justifie pour la plupart des colonnes : un revenu non déclaré est considéré par le banquier comme un revenu nul, une ancienneté nulle dans l'emploi est équivalente à ses yeux à une absence d'emploi.  

En revanche, concernant la variable d'ancienneté du véhicule par exemple, le choix de mettre la variable NaN à zéro met à égalité l'emprunteur qui ne possède pas de voiture, et celui qui fait sa demande de prêt justement le jour où il vient de prendre possession d'un nouveau véhicule : ce qui correspond à des situations patrimoniales différentes. Toutefois, cette information est complétée par le booléen "OWN_CAR_FLAG", qui permet de faire la différence.

On complète `clean_values()` :

In [ ]:
def clean_values(df):
    print('Nettoyage des lignes...')
    
    old_length = df.shape[0]
    result = df.copy()
    num_cols = [col for col in result.columns if result[col].dtype!='object']
    cat_cols = [col for col in result.columns if col not in num_cols]
    
    high_z_score = result[num_cols].apply(z_score, axis=0, args=[80])
    result.drop(high_z_score[high_z_score.any(axis=1)].index, inplace=True)
    
    days_cols = [col for col in result.columns if "DAYS_" in col]
    for col in day_cols:
        result.loc[result[col]< result['DAYS_BIRTH'], col] = result['DAYS_BIRTH']
        result.loc[result[col]>0, col] = 0
        
    result[num_cols]=result[num_cols].fillna(0)
    result[cat_cols]=result[cat_cols].fillna('Not specified')
    
    new_length = result.shape[0]
    deleted_rows = old_length - new_length
    print(f'{deleted_rows} lignes ont été supprimées soit {deleted_rows/old_length:.2%} du total.')
  
    return result

## 3. Premier essai d'entraînement

Dans un premier temps, on supprime les valeurs aberrantes, sans modifier les variables existantes :

In [ ]:
data = clean_values(application_train)

Préalable : extraction des variables et des données cible

In [ ]:
def repartition_dataset(data,target,test_size=0.2,random_state=0,stratify_target=False,under_sampling=False,over_sampling=False):
    if stratify_target:
        train_set,test_set = train_test_split(data,test_size=test_size,random_state=random_state,stratify=data[target])
    else:
        train_set,test_set = train_test_split(data,test_size=test_size,random_state=random_state)
        
    # test set
    X_test = test_set.drop(target,axis=1)
    y_test = test_set[[target]]
    print("Test =>\nX :", X_test.shape, "\ny :",y_test.shape)
    if under_sampling :
        train_set = fair_data(train_set,target)
    X_train = train_set.drop(target,axis=1)
    y_train = train_set[[target]]
    
    if over_sampling :
        over = SMOTE(sampling_strategy=0.1)
        under = RandomUnderSampler(sampling_strategy=0.5)
        steps = [('o', over), ('u', under)]
        pipeline = Pipeline(steps=steps)
        X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    print("Train =>\nX :", X_train.shape, "\ny :",y_train.shape)
    
    return X_train,X_test,y_train,y_test

In [ ]:
features_cols = [col for col in data.columns if col not in ['SK_ID_CURR']]
X = data[features_cols]
y = data['TARGET']
id = data['SK_ID_CURR']
X.shape, y.shape

In [ ]:
X_train,X_test,y_train,y_test = repartition_dataset(data[features_cols],"TARGET")

### 3.1 Choix de la métrique

Nous cherchons à établir un modèle de classification binaire. Le résultat peut donc être positif (cas du client qui est en défaut de paiement) ou négatif (le client rembourse normalement).

Un faux positif est un emprunteur qui serait à tort estimé comme présentant un risque de défaut.  
Un faux négatif est un emprunteur qui serait estimé à tort comme ne présentant pas de risque de défaut.

L'objectif de l'établissement bancaire est de détecter le plus possible des cas positifs (sensibilité élevée), et aussi de minimiser le taux de faux négatifs : en effet, cela voudrait dire qu'elle accorde un prêt à un client qui ne le remboursera pas. 

Pour autant, le taux de faux positifs n'est pas à négliger non plus car il représente pour un établissement une perte d'opportunité, en la personne d'un emprunteur qui aurait été solvable et auquel il aurait été possible de vendre d'autres produits et services.

Le taux de faux positifs est estimé par la précision (nombre total de vrais positifs divisé par le nombre total de positifs prédits). Le taux de faux négatifs est estimé par 1 - recall (le recall, ou sensibilité, étant égal au nombre de vrais positifs rapporté au nombre total de cas positifs).

L'optimisation entre la précision et le recall est donnée par la la maximisation du **F1-score**. Il s'agit de la moyenne harmonique de ces deux valeurs. Toutefois ce score attribue le même poids à ces derniers. Or nous souhaitons quant à nous privilégier la limitation des faux négatifs.

On utilisera donc le **F2-score** (c'est-à-dire un F-beta avec un Beta égal à 2), qui surpondère les faux négatifs ([source](https://machinelearningmastery.com/fbeta-measure-for-machine-learning/)).

.  Precision : $\frac{TP}{TP + FP}$

.  Recall : $\frac{TP}{TP + FN}$

.  FBeta-score ($F_{\beta}$) : $F_{\beta} = (1 + \beta^{2}) \frac{precision * recall}{\beta^{2} * precision + recall}$ 



> $TP$ => True Positif  
> $FP$ => False Postif <br>
> $FN$ => False Negatif

### 3.2 Base de comparaison : classifieur naïf

In [ ]:
ftwo_scorer = make_scorer(fbeta_score, beta=2)

def evaluation(model,X_test,y_test):
    y_pred = model.predict(X_test)
    matrix_pred_model(y_test,y_pred)
    score = ftwo_scorer(model,X_test,y_test)
    print("Score (fbeta 2):",score )
    return score

def all_eval(model,X_test,y_test):
    y_pred = model.predict(X_test)
    
    recall = recall_score(y_test,y_pred)
    precision = accuracy_score(y_test,y_pred)
    score = ftwo_scorer(model,X_test,y_test)
    
    print(classification_report(y_test,y_pred))
    return recall, precision, score

def matrix_pred_model(y_test,y_pred,figsize=(5,5)):
    plt.figure(figsize=figsize)
    plt.title("Matrice de confusion")
    sns.heatmap(confusion_matrix(y_test,y_pred),annot=True,fmt="d",cbar=False)
    plt.xlabel("Prédiction")
    plt.ylabel("Classe initial")
    plt.show()

def plot_all_roc_curve(all_model,X_test,y_test,figsize=(9,7),title_roc_curve="ROC Curve",naive_model=None):
    fig, ax = plt.subplots(figsize=figsize)
    model_displays = {}
    for name, pipeline in all_model.items():
        model_displays[name] = plot_roc_curve(pipeline, X_test, y_test, ax=ax, name=name)
        
    if not naive_model is None:
        model_displays["Naive"] = plot_roc_curve(naive_model, X_test, y_test, ax=ax, name="Naive")
        
    _ = ax.set_title(title_roc_curve)

On établit un dummy classifier qui servira de point de comparaison pour évaluer l'efficacité du modèle. Pour le choix du type de dummy classifier, vérifions la répartition des classes dans les données :

In [ ]:
sum(y)/len(y)

Comme on pouvait s'y attendre, les cas de défaut sont très minoritaires (8,07%). Un modèle qui prédirait systématiquement la classe négative (pas de défaut de paiement, l'emprunteur paie normalement) aura donc une accuracy de 91,97%.

On choisit un Dummy Classifier qui renvoie des prédictions avec la même distribution que le jeu d'entraînement, afin d'avoir la même sensibilité (taux de vrais positifs) et la même spécificité (taux de vrais négatifs).

In [ ]:
## NAIVE
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(X_train,y_train)

f2_dum = evaluation(dummy_clf,X_test,y_test)
acc_dum = dummy_clf.score(X_test, y_test)

In [ ]:
# dum = DummyClassifier(strategy='stratified')

In [ ]:
# _ = dum.fit(X, y)

L'accuracy de ce classifieur est de :

In [ ]:
# acc_dum = dum.score(X, y)
# acc_dum

Et son F2 score est de :

In [ ]:
# y_pred_dum = dum.predict(X)
# f2_dum = fbeta_score(y, y_pred_dum, beta=2)
# f2_dum

Notre modèle doit donc surperformer ce classifieur naïf.

Construisons un tableau récapitulatif pour suivre l'évolution des scores selon les progrès de la modélisation.

In [ ]:
recap = pd.DataFrame(columns=['accuracy', 'F2-score'])

In [ ]:
def update_recap(index, acc, f2, recap=recap):
    '''Add line to recap ith acuracy and f2 score.'''
    if index not in recap.index:
        recap.loc[index] = [acc, f2]
    return recap.style.format('{:.2%}')

In [ ]:
update_recap('dummy', acc_dum, f2_dum)

### 3.3 Standardisation et encodage des valeurs

La transformation des données présente deux contraintes :  
* d'une part, les données sont de nature différente (quantitative ou qualitative), et parmi les données quantitatives, certaines sont déjà normalisées ou binaires, elles nécessitent donc des traitements différents (standardisation - ou pas - ou encodage);  
* d'autre part, la déséquilibre des classes rend d'autant plus nécessaire la séparation des jeux de test, et le recours à un stratified k-fold. Or, pour éviter toute fuite d'information, la séparation des jeux de données doit être effectuée avant la normalisation.
    
Le recours à un pipeline va permettre d'effectuer la normalisation après chaque stratification.

La standardisation sera réalisée avec StandardScaler, l'encodage par One Hot Encoder.

In [ ]:
num_cols = [col for col in X_train.columns if data[col].dtype != 'object']
cat_cols = [col for col in X_train.columns if col not in num_cols]

transformer = make_column_transformer(
    (StandardScaler(), num_cols),
    (OneHotEncoder(), cat_cols)
    )

On vérifie que le transformer produit le résultat attendu (sans modifier X) :

In [ ]:
result = transformer.fit_transform(X_train)
transformed_cols = num_cols + list(transformer.named_transformers_['onehotencoder'].get_feature_names())
df_result = pd.DataFrame(result, columns=transformed_cols)
print(df_result.shape)
df_result.describe()

In [ ]:
df_result.head()

Les données sont prêtes à être fournies au pipeline du modèle. 

### 3.4 Entraînement

Pour pouvoir utiliser une stratégie d'optimisation KStratifiedDFolds, on utilisera la fonction GridSearchCV de sckit-learn. Elle permet :
-  d'éviter une fuite d'information entre jeux d'entraînement et de test, en les séparant avant normalisation
- d'entraîner le modèle, pour pouvoir ensuite examiner les résultats (là où une simple cross_validation ne ferait que donner les scores pour chaque kfold).

De plus, elle sera également applicable pour la sélection des hyperparamètres (qui est sa fonction première). Enfin, elle permet la parallélisation.

Commençons avec un DecisionTreeClassifier avec les données simplement nettoyées des valeurs aberrantes, et avec les hypermaramètres par défaut :

In [ ]:
pipeline = make_pipeline(transformer, DecisionTreeClassifier())
params = {'decisiontreeclassifier__max_depth': [None]}

ftwo_scorer = make_scorer(fbeta_score, beta=2)
accuracy_scorer = make_scorer(accuracy_score)

grid_dtc = GridSearchCV(
    pipeline, 
    param_grid=params, 
    cv=5, 
    scoring={'acc': accuracy_scorer, 'ftwo': ftwo_scorer},
    refit='ftwo', 
    verbose=2
)

In [ ]:
_ = grid_dtc.fit(X_train, y_train)

In [ ]:
# best_estimator = grid_dtc.best_estimator_
# _ = best_estimator.fit(X_train, y_train)

In [ ]:
f2_dtc = evaluation(grid_dtc,X_test,y_test)

In [ ]:
plot_all_roc_curve({"Decision Tree":grid_dtc},X_test,y_test,naive_model=dummy_clf)

In [ ]:
# update_recap('dtc', acc_dtc, f2_dtc)

In [ ]:
# plt.figure(figsize=(20,16)) 
# _=plot_tree(best_estimator.named_steps['decisiontreeclassifier'], 
#             max_depth=4, feature_names=transformed_cols, 
#             proportion=True, filled=True, fontsize=8)

On remarque que ce sont les éléments 'EXT_SOURCE' qui semblent les plus significatifs. Mais les résultats, bien que meilleurs que ceux du classifieur naïf, ne sont pas bons pour autant.

In [ ]:
update_recap('dtc basique', grid_dtc.cv_results_['mean_test_acc'][0], grid_dtc.cv_results_['mean_test_ftwo'][0])

## 4. Amélioration du modèle

- modification des variables
- modification des hyperparamètres
- changement de modèle

### 4.1 Amélioration des variables

Quelques constats préliminaires :

- le revenu ne peut être que d'une seule catégorie, alors qu'il est tout à fait possible d'avoir à la fois un salaire et des revenus immobiliers par exemple.
- la variable FLAG_WORK_PHONE est indiquée comme correspondant à l'existence du numéro de téléphone du domicile dans le dossier, alors que le nom est plus évocateur d'un numéro professionnel, d'autant qu'il existe aussi la variable FLAG_PHONE pour le domicile.
- il existe deux variables liées au numéro professionnel : FLAG_WORK_PHONE et FLAG_EMP_PHONE. Peut-être sont-elles différenciées pour traiter séparément les cas d'un entrepreneur indépendant / profession libérale et les cas de salariés, ou pour différencier la ligne directe professionnelle de celle de l'établissement, mais cela semble une lourdeur inutile.
- la variable FLAG_MOBIL doublonne avec FLAG_CONT_MOBIL : l'important n'est pas d'avoir donné un numéro mais qu'il soit effectivement joignable.
- Les variables EXT_SOURCE et FLAG_DOCUMENT ne permettent pas de vérifier que les sources et documents en question sont toujours fournis dans l'ordre attendu. Il conviendrait de vérifier ce point auprès de l'établissement bancaire. Si celui-ci n'est pas en mesure de certifier que c'est bien le cas, ces variables pourraient très certainement être regroupées.
- bien que la magie de la Data Science soit justement de faire ressortir des corrélations cachées, on ne peut s'empêcher de se demander dans quelle mesure la nature des matériaux de construction, le nombre d'entrées ou d'étages du logement de l'emprunteur, pourraient conditionner sa capacité à rembourser le prêt.
- les données relatives au cercle relationnel du client donnent à penser qu'il est licite et courant de demander ce type d'information aux USA, alors qu'en France cela constituerait une infraction à la fois au secret bancaire et au RGPD. Ce qui pose au passage la question de l'exportabilité d'un modèle de ML d'un pays à l'autre.

Examinons les corrélations entre les variables :

In [ ]:
def show_correlation_matrix(corr):
    '''display correlation matrix.'''
    fig = px.imshow(corr, title = f"Corrélation entre les variables", 
                    labels={'color':"Corrélation"}, 
                    color_continuous_scale='RdBu',
                    color_continuous_midpoint=0,
                    width=1100, height=1100, )
    fig.show()

In [ ]:
data.describe()

In [ ]:
corr = data.corr()
show_correlation_matrix(corr)

On remarque que certaines corrélations sont à NaN, ce qui correspond à des variables qui ont une seule valeur pour toutes les données (toutes ces variables sont binaires) :

In [ ]:
data[list(corr[corr['AMT_INCOME_TOTAL'].isna()].index)].describe()

Ceci est dû à la suppression des lignes présentant des outliers dans d'autres catégories. Ces variables ne sont donc d'aucune utilité pour entraîner le modèle et peuvent être supprimées.

In [ ]:
constant_cols = corr[corr['AMT_INCOME_TOTAL'].isna()].index.to_list()
data.drop(constant_cols, axis=1, inplace=True)

In [ ]:
show_correlation_matrix(data.corr())

Remarquons ensuite que le résultat (TARGET) ne semble nettement corrélé linéairement à aucune variable en particulier. Voici ci-après les corrélations, positives ou négatives, classées par ordre décroissant de valeur absolue :

In [ ]:
pd.DataFrame(corr['TARGET'].sort_values(ascending=False, key=lambda x: np.abs(x))).head(15)

Les données relatives au logement sont fortement corrélées entre elles et il semble donc possible de les synthétiser. Dans un premier temps, on peut déjà supprimer le mode et la médiane pour ne conserver que la moyenne. Cette suppression sera intégrée à la fonction `clean_features()`.

In [ ]:
columns_without_mode_and_medi = [col for col in data.columns if not (col.endswith('_MODE') or col.endswith('_MEDI'))]
data = data[columns_without_mode_and_medi]

Revisualisons la matrice de corrélation après suppression de ces éléments :

In [ ]:
show_correlation_matrix(data.corr())

Maintenant que le nombre de variables a été sensiblement réduit, examinons les corrélations supérieures (en valeur absolue) à 0.7 :

In [ ]:
def list_high_correlations(df, threshold):
    '''Print pairs of correlated features in corr and their correlation score.'''
    corr = df.corr()
    limit = 0
    list_of_corr = []
    for row in corr.index:
        for col in corr.columns[limit:]:
            if threshold<np.abs(corr.loc[row, col])<1:
                list_of_corr.append(((row, col), corr.loc[row, col]))
        limit+=1
        list_of_corr = sorted(list_of_corr, key= lambda x: np.abs(x[1]), reverse=True)
    if not list_of_corr:
        print(f'Aucune corrélation supérieure à {threshold}.')
    else:
        for element in list_of_corr:
            print(element[0], ":", f'{element[1]:.2%}')
    

In [ ]:
list_high_correlations(data, 0.7)

Décidons maintenant du traitement de ces variables. Le coefficient de corrélation est arrondi à 2 décimales.

**('DAYS_EMPLOYED', 'FLAG_EMP_PHONE') : -1**

La corrélation inverse, arrondie, est totale. Ce qui semble logique puisque dès lors que le demandeur de prêt a un employeur, il pourra fournir son numéro. Cette dernière information est donc en soi redondante (elle correspond peu ou prou à "a un employeur") et **la variable FLAG_EMP_PHONE peut être supprimée**, tandis que la variable concernant l'ancienneté dans l'emploi est beaucoup plus intéressante (c'est une variable attentivement étudiée par un banquier).

In [ ]:
data.drop(['FLAG_EMP_PHONE'], axis=1, inplace=True)

**('OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE') : 1**  
**('DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE') : 0.86**

Traçons le nuage de points pour les retards de paiement (pour une meilleure lisibilité, on ignorera la valeur aberrante de 347, on reviendra sur cette valeur dans la 2ème partie du traitement des données).

In [ ]:
px.scatter(data[data['OBS_30_CNT_SOCIAL_CIRCLE']<50], x='OBS_30_CNT_SOCIAL_CIRCLE', y='OBS_60_CNT_SOCIAL_CIRCLE')

La corrélation étant parfaite, on peut choisir d'ignorer une des deux variables. Reste à savoir laquelle. Etudions les variables relatives aux défaut de paiement :

In [ ]:
px.scatter(data[data['DEF_30_CNT_SOCIAL_CIRCLE']<50], x='DEF_30_CNT_SOCIAL_CIRCLE', y='DEF_60_CNT_SOCIAL_CIRCLE')

Reprenons les définitions de DEF_30 et DEF_60 : "How many observation of client's social surroundings defaulted on [30]/[60] (days past due) DPD". Cette variable recense le nombre de défauts de paiement dans l'entourage de l'emprunteur suite à un retard de paiement de 30 ou 60 jours. On constate sur le graphe que DEF60 n'est jamais supérieur à DEF 30, ce qui est logique : le nombre de personnes en défaut de paiement depuis plus d'un mois comprend celui en défaut de paiement depuis plus de deux mois (mais pas l'inverse).

On va donc choisir de retenir DEF_30, et par conséquent OBS_30 également. On supprimera donc les variables DEF_60 et OBS_60.

In [ ]:
data.drop(['OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE'], axis=1, inplace=True)

**('AMT_CREDIT', 'AMT_ANNUITY') 0.77**

Le montant de l'annuité croît linéairement dans le même sens que celui du crédit bien sûr, mais il dépend également de deux autres facteurs : le taux et la durée du prêt. Or ces deux facteurs pourraient aboutir à un paradoxe de Simpson. En effet, le taux du prêt est décidé par le banquier en fonction de plusieurs paramètres, dont la durée du prêt et la motivation de l'établissement bancaire à accorder le prêt. Le taux du prêt dépend donc de la qualité du dossier de l'emprunteur aux yeux de la banque. Or, pour une banque, un emprunteur de qualité, c'est un emprunteur qui ne fait pas défaut : c'est justement l'objectif du modèle que de le prédire ! 

Quant à la durée du prêt, elle impacte très fortement le montant de l'échéance et joue à ce titre un rôle majeur dans l'endettement. Il suffit parfois de la moduler pour que l'endettement passe d'excessif à acceptable.

Les banques françaises utilisent couramment deux autres variables pour mesurer le risque client : le taux d'endettement (rapport entre le total des échéances à payer et les revenus) et le reste à vivre (différence entre les échéances à payer, majorées de charges fixe de type loyer, et les revenus). Mais cela nécessite de commencer par calculer le montant de l'échéance du prêt envisagé, donc à refaire tourner le modèle pour chaque durée considérée. 

Une variable plus pertinente dans ce cas est le **rapport entre le montant du crédit et le total des revenus** : il donne une idée, certes approximative, de la mesure dans laquelle le bien est dans les moyens de l'acquéreur. Par exemple, si on retient une limite d'endettement de 33%, cela signifie qu'un prêt remboursé sans encombre pendant 15 ans représentait environ 5 ans de salaire (hypothèse très simplifiée ici puisqu'on ne tient pas compte du taux du prêt). Acheter un bien qui vaut 5 ans de salaire est donc une opération raisonnablement envisageable. Par contre, acheter un bien représentant 25 ans de salaire impliquerait de devoir s'endetter à hauteur de 33% de ses revenus pendant plus de 75 ans.  

**On créera donc la variable "CREDIT_INCOME_RATIO" et on supprimera la variable AMT_ANNUITY** qui est non seulement inutile mais dangereuse pour la fiabilité du modèle.

In [ ]:
data['CREDIT_INCOME_RATIO'] = data['AMT_CREDIT'] / data['AMT_INCOME_TOTAL']
data.drop(['AMT_ANNUITY'], axis=1, inplace=True)

**('AMT_CREDIT', 'AMT_GOODS_PRICE') 0.99**  
**('AMT_ANNUITY', 'AMT_GOODS_PRICE') : 0.78**

Là encore, la corrélation parfaite observée  entre le montant du crédit et celui du bien est évidente et a beaucoup moins d'intérêt que le **ratio** entre le montant du crédit et celui de la valeur du bien. C'est le ratio d'apport personnel, également scruté de près par les banquiers, du moins en France. Couplé à CREDIT_INCOME_RATIO, il donne là aussi une bonne idée de la faisabilité du dossier. On créera donc la variable **'CREDIT_GOOD_RATIO' qui sera égale à AMT_CREDIT / AMT_GOODS_PRICE, et on supprimera la variable 'AMT_CREDIT'** puisqu'elle corrélée à la fois a CREDIT_GOOD_RATIO et à CREDIT_INCOME_RATIO.

Comme vu plus haut, la varable AMT_ANNUITY est supprimée, et on pourra également **supprimer la valeur d'achat du bien AMT_GOODS_PRICE**, puisqu'elle peut être reconstituée à partir de AMT_INCOME, CREDIT_INCOME_RATIO et CREDIT_GOOD_RATIO.

In [ ]:
data['CREDIT_GOOD_RATIO'] = data['AMT_CREDIT']/(data['AMT_GOODS_PRICE']+1)
data.drop(['AMT_CREDIT', 'AMT_GOODS_PRICE'], axis=1, inplace=True)

**('REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY') 0.95**

La forte corrélation s'explique par le fait que l'opinion d'un établissement sur la région dont provient le client est influencée, notamment, par les villes qui en font partie. L'exclusion de la ville ne semble donc pas un choix significatif et la variable correspondante, **REGION_RATING_CLIENT_W_CITY, paraît pouvoir être omise dans le modèle**.

In [ ]:
data.drop(['REGION_RATING_CLIENT_W_CITY'], axis=1, inplace=True)

**('APARTMENTS_AVG', 'ELEVATORS_AVG') 0.84  
('APARTMENTS_AVG', 'LIVINGAPARTMENTS_AVG') 0.94  
('APARTMENTS_AVG', 'LIVINGAREA_AVG') 0.91  
('ELEVATORS_AVG', 'LIVINGAPARTMENTS_AVG') 0.81  
('ELEVATORS_AVG', 'LIVINGAREA_AVG') 0.87**  
On constate que la variable APARTMENTS_AVG présente des corrélations élevées avec les variables ELEVATORS_AVG, LIVINGAPARTMENTS_AVG et LIVING_AREA_AVG : on peut donc supprimer ces dernières. En pratique, on peut d'ailleurs douter de la pertinence, voire de l'utilité de ces éléments dont on ne connaît pas le mode de détermination.

In [ ]:
data = data.drop(['ELEVATORS_AVG', 'LIVINGAPARTMENTS_AVG','LIVINGAREA_AVG'], 
                       axis=1)

**('CNT_CHILDREN', 'CNT_FAM_MEMBERS') 0.88**

La corrélation entre le nombre d'enfants et le nombre de membres de la famille n'est pas surprenante, puisque les enfants forment un sous-ensemble des membres de la famille. Examinons-la :

In [ ]:
px.scatter(data, x=data['CNT_CHILDREN'], y=data['CNT_FAM_MEMBERS'])

Le nombre de membres de la famille correspond donc au(x) parent(s) du foyer, et c'est la statistique avec le nombre d'enfants qui est la plus pertinente. L'existence d'un lien de couple étant déjà traité par la donnée NAME_FAMILY_STATUS, la variable CNT_FAMILY_MEMBER peut être supprimée.

In [ ]:
data.drop(['CNT_FAM_MEMBERS'], axis=1, inplace=True)

**('REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION') 0.86**  
**('REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY') 0.83**

Il s'agit ici de vérifier, au niveau de la région ou de la ville, si l'adresse de travail de l'emprunteur est différente de son adresse de résidence ou de son adresse de contact. Notons que les corrélations entre les variables croisées, par exemple entre REG_REGION_NOT_WORK_REGION et REG_CITY_NOT_WORK_CITY, sont inférieures à 0.7 puisqu'elles ne ressortent pas dans le détail ci-dessus. 
On peut aussi relever là encore la différence entre un modèle américan et français puisque le premier accepte une troisième adresse en plus de celle du domicile et celle du travail : en France, l'adresse du domicile est obligatoirement l'adresse de contact, une autre adresse ne serait tout simplement pas prise en compte.

On retient ici les variables LIVE_ et on supprime les variables REG_.

In [ ]:
data = data.drop(['REG_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_WORK_CITY'], axis=1)

**('FLOORSMAX_AVG', 'FLOORSMIN_AVG') : 0.74**  
Ces données manquent de clarté : le nombre maximum d'étages concerne-t-il le logement de l'emprunteur (auquel cas c'est tout simplement le nombre d'étages) ou le type de logement qu'il occupe ? Or dans ce cas il ne s'agit pas d'une donnée qui lui est personnelle et donc susceptible d'être utile au modèle. Pour diminuer le nombre de variables on retiendra la moyenne de ces deux valeurs. On sera par ailleurs attentif à la pertinence de la variable dans le modèle.

In [ ]:
data['FLOORS_AVG'] = (data['FLOORSMAX_AVG'] + data['FLOORSMIN_AVG'])/2
data.drop(['FLOORSMAX_AVG','FLOORSMIN_AVG'], axis=1, inplace=True)

In [ ]:
def analyse_num_with_target_bin(positive_df,negative_df,target_var,remove=["Col_ID"]):
    num_cols = [col for col in positive_df.columns if not col in positive_df.select_dtypes(["object","category"]).columns.to_list()]
    if len(remove) > 0:
        num_cols = [col for col in num_cols if (col not in remove)]
        
    for col in num_cols:
        fig,axes = plt.subplots(1,3,figsize=(25,6))
        sns.distplot(positive_df[col],label="positive",ax=axes[0])
        sns.distplot(negative_df[col],label="negative",ax=axes[0])

        pd.concat([positive_df,negative_df],axis=0).boxplot(str(col),by=target_var,ax=axes[1],vert=False)
        
        pd.concat([positive_df,negative_df],axis=0).boxplot(str(col),ax=axes[2],vert=False)

        axes[0].legend()
        # Taille max du nom de la colonne 50
        axes[0].set_title(col[:50] + " | Distplot")
        axes[1].set_title(col[:50] + " | Boxplot split")
        axes[2].set_title(col[:50] + " | Boxplot global")
        
        axes[0].set_xlabel("")
        axes[1].set_xlabel("")
        axes[2].set_xlabel("")
        
        axes[2].set_yticklabels([])
        axes[2].set_yticks([])
        

        plt.show()

In [ ]:
new_col = ["CREDIT_INCOME_RATIO","CREDIT_GOOD_RATIO","FLOORS_AVG"]
positive_target = data[data["TARGET"] == 1].sample(20000,random_state=0)
negative_target = data[data["TARGET"] == 0].sample(20000,random_state=0)
analyse_num_with_target_bin(positive_target,negative_target,"TARGET",remove=[col for col in data.columns if not col in new_col])

Affichons la matrice de corrélation actualisée :

In [ ]:
show_correlation_matrix(data.corr())

In [ ]:
list_high_correlations(data, 0.7)

APARTMENTS_AVG est fortement corrélé à 3 variables : ENTRANCES_AVG, BASEMENTAREA_AVG et FLOORS_AVG, on supprime celles-ci.

In [ ]:
data.drop(['ENTRANCES_AVG', 'BASEMENTAREA_AVG', 'FLOORS_AVG'], axis=1, inplace=True)

In [ ]:
list_high_correlations(data, 0.5)

Les corrélations ne semblent pas suffisantes pour autoriser une modification des variables. 

Vérifions les données après nettoyage :

In [ ]:
data.describe()

On intègre les opérations réalisées ci-dessus à la fonction `clean_features()` :

In [ ]:
def clean_features(df):
    print('Nettoyage des colonnes...')
    # Drop columns with a single value
    
    corr = df.corr()
    constant_cols = corr[corr['AMT_INCOME_TOTAL'].isna()].index.to_list()
    df.drop(constant_cols, axis=1, inplace=True)
    
    # Drop MODE and MEDI columns 
    
    cols_without_mode_and_medi = [col for col in df.columns 
                                     if not (col.endswith('_MODE') or col.endswith('_MEDI'))]
    df = df[cols_without_mode_and_medi]
    
    # Create new synthetic features
    
    df['CREDIT_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
    df['CREDIT_GOOD_RATIO'] = df['AMT_CREDIT']/(df['AMT_GOODS_PRICE']+1)
    df['FLOORS_AVG'] = (df['FLOORSMAX_AVG'] + df['FLOORSMIN_AVG'])/2
    
    # Drop redundant columns
    
    cols_to_drop = ['FLAG_EMP_PHONE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE',
                   'AMT_ANNUITY','AMT_CREDIT', 'AMT_GOODS_PRICE', 'REGION_RATING_CLIENT_W_CITY',
                   'ELEVATORS_AVG', 'LIVINGAPARTMENTS_AVG','LIVINGAREA_AVG','CNT_FAM_MEMBERS',
                    'REG_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_WORK_CITY',
                   'FLOORSMAX_AVG','FLOORSMIN_AVG',
                    'ENTRANCES_AVG', 'BASEMENTAREA_AVG', 'FLOORS_AVG']
    
    df = df.drop(cols_to_drop, axis=1)   
        
    return df

In [ ]:
cleaned_values_data = clean_values(application_train)
data = clean_features(cleaned_values_data)

In [ ]:
data.describe()

In [ ]:
features_cols = [col for col in data.columns if col not in ['SK_ID_CURR', 'TARGET']]
X = data[features_cols]
y = data['TARGET']
id = data['SK_ID_CURR']
X.shape, y.shape

In [ ]:
X_train,X_test,y_train,y_test = repartition_dataset(data[features_cols],"TARGET")

La standardisation sera réalisée avec StandardScaler, l'encodage par One Hot Encoder.

In [ ]:
num_cols = [col for col in X_train.columns if data[col].dtype != 'object']
cat_cols = [col for col in X_train.columns if col not in num_cols]

transformer = make_column_transformer(
    (StandardScaler(), num_cols),
    (OneHotEncoder(), cat_cols)
    )

On vérifie que le transformer produit le résultat attendu (sans modifier X) :

In [ ]:
result = transformer.fit_transform(X_train)
transformed_cols = num_cols + list(transformer.named_transformers_['onehotencoder'].get_feature_names())
df_result = pd.DataFrame(result, columns=transformed_cols)
print(df_result.shape)
df_result.describe()

### 4.2 Essai avec variables modifiées

On relance le modèle après avoir actualisé le transformer chargé d'encoder X puisque les variables ont changé : 

In [ ]:
num_cols = [col for col in X_train.columns if data[col].dtype != 'object']
cat_cols = [col for col in X_train.columns if col not in num_cols]

transformer = make_column_transformer(
    (StandardScaler(), num_cols),
    (OneHotEncoder(), cat_cols)
    )

pipeline = make_pipeline(transformer, DecisionTreeClassifier())
params = {'decisiontreeclassifier__max_depth': [None]}
ftwo_scorer = make_scorer(fbeta_score, beta=2)
accuracy_scorer = make_scorer(accuracy_score)

grid_dtc_n = GridSearchCV(
    pipeline, 
    param_grid=params, 
    cv=5, 
    scoring={'acc': accuracy_scorer, 'ftwo': ftwo_scorer},
    refit='ftwo', 
    verbose=2
)
_= grid_dtc_n.fit(X_train,y_train)

In [ ]:
f2_dtc_n = evaluation(grid_dtc_n,X_test,y_test)

In [ ]:
plot_all_roc_curve({"Decision Tree New Var":grid_dtc_n},X_test,y_test,naive_model=dummy_clf)

In [ ]:
update_recap('dtc nouvelles variables', 
             grid_dtc_n.cv_results_['mean_test_acc'][0], 
             grid_dtc_n.cv_results_['mean_test_ftwo'][0])

Le traitement des variables n'améliore pas la performance du modèle mais la temps de execution est réduit à moitié

### 4.3 Essai avec régression logistique

Essayons un autre modèle, la régression logistique :

In [ ]:
pipeline = make_pipeline(transformer, LogisticRegression())
params = {'logisticregression__C': [.1],
         'logisticregression__max_iter': [10],
         'logisticregression__solver': ['saga']}

grid_lr = GridSearchCV(
    pipeline, 
    param_grid=params, 
    cv=5, 
    scoring={'acc': accuracy_scorer, 'ftwo': ftwo_scorer},
    refit='ftwo', 
    verbose=2
)

In [ ]:
_ = grid_lr.fit(X_train,y_train)

In [ ]:
f2_lr = evaluation(grid_lr,X_test,y_test)

In [ ]:
# plot_all_roc_curve({"logistique régression":grid_lr},X_test,y_test,naive_model=grid_dtc)

In [ ]:
update_recap('régression logistique',
             grid_lr.cv_results_['mean_test_acc'][0], 
             grid_lr.cv_results_['mean_test_ftwo'][0])

In [ ]:
grid_lr.best_params_

### 4.4 Essai avec Random Forest

In [ ]:
pipeline = make_pipeline(transformer, RandomForestClassifier())

In [ ]:
# sorted(pipeline.get_params().keys())

In [ ]:

params = {'randomforestclassifier__n_estimators': [250],
          'randomforestclassifier__class_weight': ['balanced'],
          'randomforestclassifier__criterion': ["entropy"],
          'randomforestclassifier__max_depth': [10],
          'randomforestclassifier__n_jobs': [50],
          'randomforestclassifier__random_state': [0]
          }

grid_rf = GridSearchCV(
    pipeline, 
    param_grid=params, 
    cv=5, 
    scoring={'acc': accuracy_scorer, 'ftwo': ftwo_scorer},
    refit='ftwo', 
    verbose=2
)

In [ ]:
_ = grid_rf.fit(X_train,y_train)

In [ ]:
f2_rf = evaluation(grid_rf,X_test,y_test)

In [ ]:
plot_all_roc_curve({"Random Forest":grid_rf, "logistique régression":grid_lr, "Decision Tree New Var":grid_dtc_n},X_test,y_test,naive_model=dummy_clf)

In [ ]:
update_recap('Random Forest',
             grid_rf.cv_results_['mean_test_acc'][0], 
             grid_rf.cv_results_['mean_test_ftwo'][0])

In [ ]:
grid_rf.best_params_

In [ ]:
grid_rf.predict(X_test)

## Entrainement sur tout le jeu de données

Maintenant que nous avons un modèle suffisamment performant, nous allons l'entraîner sur la totalité du jeu de données pour permettre de faire des prédictions les plus fiables possibles sur des données extérieures.

In [ ]:
best_model = RandomForestClassifier(n_estimators=250,
                                    criterion="entropy",
                                    max_depth=10,
                                    class_weight="balanced",
                                    random_state=0,n_jobs=50)

In [ ]:
feature_names = [i for i in X_train.columns if X_train[i].dtype in [np.float64, np.float32, np.int64, np.int32]]
df_features = X_train[feature_names]
print("Shape X :",df_features.shape)
print("Shape y :",y_train.shape)

In [ ]:
feature_names

In [ ]:
from sklearn.model_selection import cross_val_score

print("Score CV :",cross_val_score(best_model,df_features,y_train,scoring=ftwo_scorer,cv=5).mean())

In [ ]:
best_model.fit(df_features,y_train)

**Le modèle est prêt à être utilisé !**

# Explication du modèle

Dans cette partie, nous allons étudier le comportement du modèle pour expliquer sa prise de décision sur les individus, pour ça, nous allons faire ressortir les paramètres les plus marquants dans la prise de décision du modèle.

In [ ]:
def get_importance_feature(model,feature,sort=True,n_first=None):
    if  hasattr(model,"coef_"):
        coef = model.coef_.tolist()[0]
    elif hasattr(model,"feature_importances_"):
         coef = model.feature_importances_.tolist()    
    importance_feature = {}
    for i in range(len(feature)):
        importance_feature[feature[i]]=abs(coef[i])
    
    if sort:
        importance_feature = dict(sorted(importance_feature.items(), key=lambda item: item[1],reverse=True))
    
    if n_first!=None:
        importance_feature={list(importance_feature.keys())[i]:list(importance_feature.values())[i] for i in range(len(importance_feature)) if i < n_first }
    
    return importance_feature
    
def get_best_param_model(importance_feature,figsize=(20,7),log=False):
    feature = list(importance_feature.keys())
    coef = list(importance_feature.values())
    plt.figure(figsize=figsize)
    plt.title("Importance des variables")
    plt.bar(feature,coef)
    plt.ylabel("coef attribué par le modèle")
    if log:
        plt.yscale("log")
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
get_best_param_model(get_importance_feature(best_model,df_features.columns,n_first=20))

## Explication SHAP 


SHAP : SHapley Additive ExPlanations

SHAP permet lui d'expliquer l'impact des variables sur le modèle ou la prédiction d'un client par rapport aux différentes variables, il permet également de voir l'impact global du jeu de données des différentes variables.

Cet approche donnent un meilleur aperçu des prédictions du modèle et l'impact des variables sur les prédictions, elles permettent également de voir à quel dégrée les variables influences une décision.

### SHAP :

In [ ]:
# !pip install shap

In [ ]:
import shap
plt.style.use("fivethirtyeight")
shap.initjs()

In [ ]:
shap_explainer = shap.TreeExplainer(best_model)

In [ ]:
%%time
shap_values = shap_explainer.shap_values(df_features[:10000]) 

In [ ]:
print('Expected Value:', shap_explainer.expected_value)

In [ ]:
shap.initjs()
shap.force_plot(shap_explainer.expected_value[1], 
                shap_values[1][0,:], df_features.iloc[0,:])

In [ ]:
shap.initjs()
shap.force_plot(shap_explainer.expected_value[1], 
                shap_values[1][1,:], df_features.iloc[1,:])

In [ ]:
# Impact pour le non remboursement class 1
shap.initjs()
shap.force_plot(shap_explainer.expected_value[1], 
                shap_values[1][:1000,:], df_features.iloc[:1000,:])

Ce graphique nous donne de multiples informations, il permet de voir les effets des décisions par rapport aux différentes variables, si on sélectionne par exemple **les ressources extérieures**, on remarque par exemple que **plus le score de ces variables est faible plus la chance de non-remboursement est élevée**. 

On peut également analyser toutes les autres variables, par exemple le **'DAYS_EMPLOYED'** on voit que les personnes qui ont **plus de 5 ans d'ancienneté ont plus de chance de rembourser leur crédit !**

In [ ]:
shap.summary_plot(shap_values,df_features[:10000])

Globalement, il est difficile de dire quelle variable impact plus une classe qu'une autre, c'est pour ça qu'il est important de voir les variantes pour la prédiction pour chaque variable. ( On choisira ici les plus impactant comme les EXT_SOURCE_{1,2,3} , DAYS_EMPLOYED, DAYS_BIRTH...)

In [ ]:
shap.dependence_plot('DAYS_EMPLOYED', shap_values[1], df_features[:10000],interaction_index=None)

Ici on voit que à partir d'environ 5 ans d'ancienneté la chance de rembourser le prêt et de plus en plus important.

In [ ]:
shap.dependence_plot('DAYS_BIRTH', shap_values[1], df_features[:10000],interaction_index=None)

Pour l'âge, au-dessus de 41 ans, il semble que les individus aient plus de chance de rembourser le prêt.

In [ ]:
shap.dependence_plot('EXT_SOURCE_1', shap_values[1], df_features[:10000],interaction_index=None)

In [ ]:
shap.dependence_plot('EXT_SOURCE_2', shap_values[1], df_features[:10000],interaction_index=None)

In [ ]:
shap.dependence_plot('EXT_SOURCE_3', shap_values[1], df_features[:10000],interaction_index=None)

Pour les ressources extérieures, la limite pour les ressources:
- 1 est à environ 0.35
- 2 est à environ 0.45
- 3 est à environ 0.45

En dessous de ces limites, les clients ont du mal à rembourser leur crédit